In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [2]:
# ============================================================
# LOAD DATA
# ============================================================
df = pd.read_csv("experiment_metrics.csv")

# Create output directory
output_dir = Path("grid_search_analysis")
output_dir.mkdir(exist_ok=True)

print("="*80)
print("GRID SEARCH ANALYSIS")
print("="*80)
print(f"Total experiments: {len(df)}")
print(f"Columns: {df.columns.tolist()[:15]}...")

GRID SEARCH ANALYSIS
Total experiments: 27
Columns: ['exp_name', 'n_blocks', 'hidden_dim', 'history', 'weather_lags', 'num_epochs', 'early_stop_epoch', 'best_val_loss', 'test_loss', 'total_time_sec', 'train_loss_ep1', 'val_loss_ep1', 'time_sec_ep1', 'train_loss_ep2', 'val_loss_ep2']...


In [3]:
# ============================================================
# 1. AGGREGATE STATISTICS BY HYPERPARAMETER
# ============================================================
print("\n" + "="*80)
print("1. AGGREGATE STATISTICS BY HYPERPARAMETER")
print("="*80)

def group_stats(df, col):
    """Compute aggregated stats for a hyperparameter."""
    stats = df.groupby(col)["best_val_loss"].agg([
        "count", "mean", "std", "min", "max", "median"
    ]).round(5)
    stats["range"] = stats["max"] - stats["min"]
    return stats.sort_values("mean")

# By history length
print("\n--- By History Length ---")
history_stats = group_stats(df, "history")
print(history_stats)

# By weather lags
print("\n--- By Weather Lags ---")
weather_stats = group_stats(df, "weather_lags")
print(weather_stats)

# By model config (n_blocks)
print("\n--- By N_Blocks ---")
blocks_stats = group_stats(df, "n_blocks")
print(blocks_stats)

# By hidden_dim
print("\n--- By Hidden Dim ---")
hidden_stats = group_stats(df, "hidden_dim")
print(hidden_stats)

# By model config combined
df["model_config"] = df["n_blocks"].astype(str) + "b_" + df["hidden_dim"].astype(str) + "h"
print("\n--- By Model Config (combined) ---")
model_stats = group_stats(df, "model_config")
print(model_stats)


1. AGGREGATE STATISTICS BY HYPERPARAMETER

--- By History Length ---
                 count     mean      std     min     max  median   range
history                                                                 
24h_hourly           9  0.04156  0.00064  0.0406  0.0424  0.0417  0.0018
24h_1week_daily      9  0.04240  0.00101  0.0412  0.0439  0.0420  0.0027
24h_1week_6h         9  0.04309  0.00128  0.0421  0.0452  0.0423  0.0031

--- By Weather Lags ---
              count     mean      std     min     max  median   range
weather_lags                                                         
sparse_24h        9  0.04151  0.00067  0.0406  0.0424  0.0415  0.0018
minimal           9  0.04199  0.00028  0.0415  0.0423  0.0420  0.0008
dense_24h         9  0.04354  0.00114  0.0419  0.0452  0.0436  0.0033

--- By N_Blocks ---
          count     mean      std     min     max  median   range
n_blocks                                                         
6             9  0.04227  0.00119  0.

In [4]:
# ============================================================
# 2. INTERACTION HEATMAPS
# ============================================================
print("\n" + "="*80)
print("2. GENERATING INTERACTION HEATMAPS")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# History × Weather Lags
pivot1 = df.pivot_table(values="best_val_loss", index="history", columns="weather_lags", aggfunc="mean")
sns.heatmap(pivot1, annot=True, fmt=".4f", cmap="RdYlGn_r", ax=axes[0, 0])
axes[0, 0].set_title("Best Val Loss: History × Weather Lags")

# History × Model Config
pivot2 = df.pivot_table(values="best_val_loss", index="history", columns="model_config", aggfunc="mean")
sns.heatmap(pivot2, annot=True, fmt=".4f", cmap="RdYlGn_r", ax=axes[0, 1])
axes[0, 1].set_title("Best Val Loss: History × Model Config")

# Weather Lags × Model Config
pivot3 = df.pivot_table(values="best_val_loss", index="weather_lags", columns="model_config", aggfunc="mean")
sns.heatmap(pivot3, annot=True, fmt=".4f", cmap="RdYlGn_r", ax=axes[1, 0])
axes[1, 0].set_title("Best Val Loss: Weather Lags × Model Config")

# Train-Val Gap (overfitting indicator)
df["train_val_gap"] = df["val_loss_ep15"] - df["train_loss_ep15"] if "val_loss_ep15" in df.columns else None
# Use last available epoch
last_epoch = df["num_epochs"].max()
df["final_train_loss"] = df[[f"train_loss_ep{i}" for i in range(1, last_epoch+1) if f"train_loss_ep{i}" in df.columns]].apply(
    lambda row: row.dropna().iloc[-1] if not row.dropna().empty else None, axis=1
)
df["final_val_loss"] = df[[f"val_loss_ep{i}" for i in range(1, last_epoch+1) if f"val_loss_ep{i}" in df.columns]].apply(
    lambda row: row.dropna().iloc[-1] if not row.dropna().empty else None, axis=1
)
df["train_val_gap"] = df["final_val_loss"] - df["final_train_loss"]

pivot4 = df.pivot_table(values="train_val_gap", index="weather_lags", columns="history", aggfunc="mean")
sns.heatmap(pivot4, annot=True, fmt=".4f", cmap="RdYlGn_r", ax=axes[1, 1])
axes[1, 1].set_title("Train-Val Gap (Overfitting): Weather Lags × History")

plt.tight_layout()
plt.savefig(output_dir / "heatmaps_interactions.png", dpi=150)
plt.close()
print(f"Saved: {output_dir / 'heatmaps_interactions.png'}")


2. GENERATING INTERACTION HEATMAPS
Saved: grid_search_analysis\heatmaps_interactions.png


In [5]:
# ============================================================
# 3. EARLY LEARNING DYNAMICS
# ============================================================
print("\n" + "="*80)
print("3. EARLY LEARNING DYNAMICS ANALYSIS")
print("="*80)

# Extract early epoch losses
early_epochs = 5
early_train_cols = [f"train_loss_ep{i}" for i in range(1, early_epochs+1)]
early_val_cols = [f"val_loss_ep{i}" for i in range(1, early_epochs+1)]

# Check for underfitting (both high) vs overfitting (train low, val high)
df["early_train_mean"] = df[early_train_cols].mean(axis=1)
df["early_val_mean"] = df[early_val_cols].mean(axis=1)
df["early_gap"] = df["early_val_mean"] - df["early_train_mean"]

# Classify learning behavior
def classify_learning(row):
    if row["early_gap"] > 0.01:
        return "Overfitting"
    elif row["early_train_mean"] > 0.06:
        return "Underfitting"
    else:
        return "Healthy"

df["learning_behavior"] = df.apply(classify_learning, axis=1)

print("\n--- Learning Behavior by Config ---")
behavior_pivot = pd.crosstab(df["weather_lags"], df["learning_behavior"])
print(behavior_pivot)

# Plot training curves comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Get epoch columns
max_ep = df["num_epochs"].max()
train_cols = [f"train_loss_ep{i}" for i in range(1, max_ep+1) if f"train_loss_ep{i}" in df.columns]
val_cols = [f"val_loss_ep{i}" for i in range(1, max_ep+1) if f"val_loss_ep{i}" in df.columns]

# Plot by weather_lags
for ax, group_col, title in zip(axes, ["weather_lags", "history", "model_config"], 
                                  ["By Weather Lags", "By History", "By Model Config"]):
    for name, group in df.groupby(group_col):
        val_losses = group[val_cols].mean()
        ax.plot(range(1, len(val_losses)+1), val_losses, label=name, marker='o', markersize=3)
    ax.set_xlabel("Epoch")
    ax.set_ylabel("Val Loss")
    ax.set_title(f"Validation Loss Curves {title}")
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / "training_curves_by_config.png", dpi=150)
plt.close()
print(f"Saved: {output_dir / 'training_curves_by_config.png'}")



3. EARLY LEARNING DYNAMICS ANALYSIS

--- Learning Behavior by Config ---
learning_behavior  Healthy
weather_lags              
dense_24h                9
minimal                  9
sparse_24h               9
Saved: grid_search_analysis\training_curves_by_config.png


In [6]:
# ============================================================
# 4. OVERFITTING ANALYSIS
# ============================================================
print("\n" + "="*80)
print("4. OVERFITTING ANALYSIS")
print("="*80)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of train-val gap by weather_lags
gap_by_weather = df.groupby("weather_lags")["train_val_gap"].agg(["mean", "std"])
gap_by_weather.plot(kind="bar", y="mean", yerr="std", ax=axes[0], color="coral", capsize=4)
axes[0].set_title("Train-Val Gap by Weather Lags (Higher = More Overfitting)")
axes[0].set_ylabel("Gap (Val - Train)")
axes[0].tick_params(axis='x', rotation=45)

# Bar chart of train-val gap by history
gap_by_history = df.groupby("history")["train_val_gap"].agg(["mean", "std"])
gap_by_history.plot(kind="bar", y="mean", yerr="std", ax=axes[1], color="steelblue", capsize=4)
axes[1].set_title("Train-Val Gap by History Length")
axes[1].set_ylabel("Gap (Val - Train)")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(output_dir / "overfitting_analysis.png", dpi=150)
plt.close()
print(f"Saved: {output_dir / 'overfitting_analysis.png'}")



4. OVERFITTING ANALYSIS
Saved: grid_search_analysis\overfitting_analysis.png


In [7]:
# ============================================================
# 5. EFFICIENCY ANALYSIS (Pareto Frontier)
# ============================================================
print("\n" + "="*80)
print("5. EFFICIENCY ANALYSIS (Val Loss vs Training Time)")
print("="*80)

fig, ax = plt.subplots(figsize=(10, 7))

# Scatter plot colored by weather_lags
colors = {"minimal": "green", "sparse_24h": "blue", "dense_24h": "red"}
markers = {"24h_hourly": "o", "24h_1week_daily": "s", "24h_1week_6h": "^"}

for _, row in df.iterrows():
    ax.scatter(row["total_time_sec"], row["best_val_loss"], 
               c=colors.get(row["weather_lags"], "gray"),
               marker=markers.get(row["history"], "o"),
               s=100, alpha=0.7)

# Add Pareto frontier
sorted_df = df.sort_values("total_time_sec")
pareto_points = []
best_val = float("inf")
for _, row in sorted_df.iterrows():
    if row["best_val_loss"] < best_val:
        pareto_points.append((row["total_time_sec"], row["best_val_loss"]))
        best_val = row["best_val_loss"]

if pareto_points:
    pareto_x, pareto_y = zip(*pareto_points)
    ax.plot(pareto_x, pareto_y, 'k--', linewidth=2, label="Pareto Frontier")

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='minimal'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='sparse_24h'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='dense_24h'),
    Line2D([0], [0], marker='o', color='gray', markersize=10, label='24h_hourly'),
    Line2D([0], [0], marker='s', color='gray', markersize=10, label='24h_1week_daily'),
    Line2D([0], [0], marker='^', color='gray', markersize=10, label='24h_1week_6h'),
]
ax.legend(handles=legend_elements, loc="upper right")

ax.set_xlabel("Total Training Time (seconds)")
ax.set_ylabel("Best Validation Loss")
ax.set_title("Efficiency: Val Loss vs Training Time\n(Color=Weather Lags, Shape=History)")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / "efficiency_pareto.png", dpi=150)
plt.close()
print(f"Saved: {output_dir / 'efficiency_pareto.png'}")



5. EFFICIENCY ANALYSIS (Val Loss vs Training Time)
Saved: grid_search_analysis\efficiency_pareto.png


In [8]:
# ============================================================
# 6. TOP CONFIGURATIONS SUMMARY
# ============================================================
print("\n" + "="*80)
print("6. TOP 10 CONFIGURATIONS")
print("="*80)

top_configs = df.nsmallest(10, "best_val_loss")[
    ["exp_name", "history", "weather_lags", "model_config", 
     "best_val_loss", "train_val_gap", "total_time_sec", "num_epochs"]
].reset_index(drop=True)

print(top_configs.to_string())



6. TOP 10 CONFIGURATIONS
                                   exp_name          history weather_lags model_config  best_val_loss  train_val_gap  total_time_sec  num_epochs
0        grid/b6_h512_24h_hourly_sparse_24h       24h_hourly   sparse_24h      6b_512h         0.0406         0.0023             693          15
1       grid/b8_h1024_24h_hourly_sparse_24h       24h_hourly   sparse_24h     8b_1024h         0.0409         0.0048            1576          15
2       grid/b10_h512_24h_hourly_sparse_24h       24h_hourly   sparse_24h     10b_512h         0.0409         0.0021            1079          15
3   grid/b6_h512_24h_1week_daily_sparse_24h  24h_1week_daily   sparse_24h      6b_512h         0.0412         0.0035             810          15
4          grid/b8_h1024_24h_hourly_minimal       24h_hourly      minimal     8b_1024h         0.0415        -0.0001            1543          15
5  grid/b8_h1024_24h_1week_daily_sparse_24h  24h_1week_daily   sparse_24h     8b_1024h         0.0415   

In [9]:
# ============================================================
# 7. RECOMMENDATIONS
# ============================================================
print("\n" + "="*80)
print("7. RECOMMENDATIONS")
print("="*80)

# Best by each criterion
best_val = df.loc[df["best_val_loss"].idxmin()]
best_efficient = df.loc[(df["best_val_loss"] / df["total_time_sec"]).idxmin()] if df["total_time_sec"].notna().any() else None
least_overfit = df.loc[df["train_val_gap"].idxmin()]

print(f"\n🏆 BEST VALIDATION LOSS:")
print(f"   Config: {best_val['exp_name']}")
print(f"   Val Loss: {best_val['best_val_loss']:.5f}")
print(f"   Gap: {best_val['train_val_gap']:.5f}")

print(f"\n⚖️ LEAST OVERFITTING:")
print(f"   Config: {least_overfit['exp_name']}")
print(f"   Val Loss: {least_overfit['best_val_loss']:.5f}")
print(f"   Gap: {least_overfit['train_val_gap']:.5f}")

# Aggregate recommendation
print("\n📊 AGGREGATE INSIGHTS:")
print(f"   • Best History: {history_stats.index[0]} (mean val: {history_stats.iloc[0]['mean']:.5f})")
print(f"   • Best Weather Lags: {weather_stats.index[0]} (mean val: {weather_stats.iloc[0]['mean']:.5f})")
print(f"   • Best Model: {model_stats.index[0]} (mean val: {model_stats.iloc[0]['mean']:.5f})")

# Save summary to file
summary_path = output_dir / "summary_report.txt"
with open(summary_path, "w") as f:
    f.write("GRID SEARCH ANALYSIS SUMMARY\n")
    f.write("="*80 + "\n\n")
    f.write("TOP 10 CONFIGURATIONS:\n")
    f.write(top_configs.to_string() + "\n\n")
    f.write("AGGREGATE STATS BY HISTORY:\n")
    f.write(history_stats.to_string() + "\n\n")
    f.write("AGGREGATE STATS BY WEATHER LAGS:\n")
    f.write(weather_stats.to_string() + "\n\n")
    f.write("AGGREGATE STATS BY MODEL CONFIG:\n")
    f.write(model_stats.to_string() + "\n\n")

print(f"\nSaved summary to: {summary_path}")
print(f"\nAll plots saved to: {output_dir}/")


7. RECOMMENDATIONS

🏆 BEST VALIDATION LOSS:
   Config: grid/b6_h512_24h_hourly_sparse_24h
   Val Loss: 0.04060
   Gap: 0.00230

⚖️ LEAST OVERFITTING:
   Config: grid/b6_h512_24h_hourly_minimal
   Val Loss: 0.04180
   Gap: -0.00140

📊 AGGREGATE INSIGHTS:
   • Best History: 24h_hourly (mean val: 0.04156)
   • Best Weather Lags: sparse_24h (mean val: 0.04151)
   • Best Model: 6b_512h (mean val: 0.04227)

Saved summary to: grid_search_analysis\summary_report.txt

All plots saved to: grid_search_analysis/
